In [1]:
from pathlib import Path
import sys
project_root = next((parent for parent in [Path.cwd()] + list(Path.cwd().parents) if (parent / "pyproject.toml").exists()), Path.cwd())
sys.path.append(str(project_root))

In [2]:
from google.cloud import bigquery

client = bigquery.Client(project="trelis-arc")

table_name = "trelis-arc.arc.shortest_ratio_2_5x_filtered_250"
file_name = table_name.split('.')[-1]

In [6]:
create_final_table_query = f"""
CREATE OR REPLACE TABLE `{table_name}` AS

WITH training_tasks AS (
    SELECT DISTINCT task_id
    FROM `trelis-arc.arc.arc_task_ids`
    WHERE subset = "arc-agi-1/training"
),
-- Clean programs by collapsing multiple empty lines into single empty lines
programs_cleaned AS (
    SELECT 
        k.task_id,
        -- Clean code by collapsing multiple consecutive newlines into at most one empty line
        -- Pattern matches multiple consecutive newlines with optional whitespace
        REGEXP_REPLACE(k.code, r'\\n(\\s*\\n)+', '\\n\\n') as code,
        k.model,
        k.predicted_train_output,
        k.predicted_test_output,
        k.correct_train_input,
        k.correct_test_input
    FROM `trelis-arc.arc.king_programs_ext` k
    INNER JOIN training_tasks t ON k.task_id = t.task_id
    WHERE k.model != 'hodel-translated'
),
-- Calculate metrics and filter by grid size
programs_with_metrics AS (
    SELECT 
        task_id,
        code,
        model,
        predicted_train_output,
        predicted_test_output,
        correct_train_input,
        correct_test_input,
        LENGTH(code) as program_length,
        -- Check if all train inputs are correct
        (SELECT LOGICAL_AND(correct_val.element) 
         FROM UNNEST(correct_train_input.list) AS correct_val) as all_train_correct,
        -- Check if all test inputs are correct
        (SELECT LOGICAL_AND(correct_val.element) 
         FROM UNNEST(correct_test_input.list) AS correct_val) as all_test_correct,
        -- Count correct examples
        (SELECT COUNTIF(correct_val.element) 
         FROM UNNEST(correct_train_input.list) AS correct_val) + 
        (SELECT COUNTIF(correct_val.element) 
         FROM UNNEST(correct_test_input.list) AS correct_val) as total_correct,
        ARRAY_LENGTH(correct_train_input.list) + ARRAY_LENGTH(correct_test_input.list) as total_possible,
        -- Check grid sizes for train output
        (SELECT MAX(ARRAY_LENGTH(grid_2d.element.list)) 
         FROM UNNEST(predicted_train_output.list) AS grid_2d) as max_train_grid_height,
        (SELECT MAX(ARRAY_LENGTH(row_1d.element.list)) 
         FROM UNNEST(predicted_train_output.list) AS grid_2d,
              UNNEST(grid_2d.element.list) AS row_1d) as max_train_grid_width,
        -- Check grid sizes for test output
        (SELECT MAX(ARRAY_LENGTH(grid_2d.element.list)) 
         FROM UNNEST(predicted_test_output.list) AS grid_2d) as max_test_grid_height,
        (SELECT MAX(ARRAY_LENGTH(row_1d.element.list)) 
         FROM UNNEST(predicted_test_output.list) AS grid_2d,
              UNNEST(grid_2d.element.list) AS row_1d) as max_test_grid_width,
        -- Normalize code for deduplication (remove all whitespace, lowercase)
        LOWER(REGEXP_REPLACE(code, r'\\s+', '')) as normalized_code
    FROM programs_cleaned
),
-- Filter by grid size (40x40) and require at least one correct
programs_filtered AS (
    SELECT *,
        -- Calculate success rate for ranking
        SAFE_DIVIDE(total_correct, total_possible) as success_rate
    FROM programs_with_metrics
    WHERE max_train_grid_height <= 40 AND max_train_grid_width <= 40
      AND max_test_grid_height <= 40 AND max_test_grid_width <= 40
      AND total_correct > 0
),
-- Find shortest, most correct program per task
task_benchmarks AS (
    SELECT 
        task_id,
        MIN(program_length) as shortest_best_length
    FROM (
        SELECT 
            task_id,
            program_length,
            ROW_NUMBER() OVER (
                PARTITION BY task_id 
                ORDER BY success_rate DESC, program_length ASC, code ASC
            ) as rank
        FROM programs_filtered
    ) ranked
    WHERE rank = 1
    GROUP BY task_id
),
-- Filter programs to those within 2.5x the shortest best program per task
programs_length_filtered AS (
    SELECT p.*
    FROM programs_filtered p
    INNER JOIN task_benchmarks b ON p.task_id = b.task_id
    WHERE p.program_length <= 2.5 * b.shortest_best_length
),
-- Deduplicate programs (same normalized code + task_id)
programs_deduplicated AS (
    SELECT 
        task_id, code, model, predicted_train_output, predicted_test_output,
        correct_train_input, correct_test_input, program_length, success_rate,
        ROW_NUMBER() OVER (
            PARTITION BY task_id, normalized_code
            ORDER BY success_rate DESC, program_length ASC, model ASC, code ASC
        ) as dedup_rank
    FROM programs_length_filtered
),
-- Take top 500 per task, prioritizing correctness then length
final_selection AS (
    SELECT 
        task_id, code, model, predicted_train_output, predicted_test_output,
        correct_train_input, correct_test_input,
        ROW_NUMBER() OVER (
            PARTITION BY task_id 
            ORDER BY success_rate DESC, program_length ASC, model ASC, code ASC
        ) as final_rank
    FROM programs_deduplicated
    WHERE dedup_rank = 1
)
SELECT task_id, code, model, predicted_train_output, predicted_test_output,
       correct_train_input, correct_test_input
FROM final_selection
WHERE final_rank <= 250
ORDER BY task_id, final_rank
"""

print("Executing BigQuery table creation...")
job = client.query(create_final_table_query)
result = job.result()
print(f"✓ Table `{table_name}` created successfully")

Executing BigQuery table creation...
✓ Table `trelis-arc.arc.shortest_ratio_2_5x_filtered_250` created successfully
✓ Table `trelis-arc.arc.shortest_ratio_2_5x_filtered_250` created successfully


In [8]:
from llm_python.datasets.bigquery_export import load_bigquery_table_as_dataframe

# Load BigQuery table as DataFrame using our reusable function
print("Loading BigQuery table data...")
raw_data = load_bigquery_table_as_dataframe(
    client=client,
    table_name=table_name
)
print(f"Loaded {len(raw_data)} programs from BigQuery table")

Loading BigQuery table data...
Exporting BigQuery table 'trelis-arc.arc.shortest_ratio_2_5x_filtered_250' to GCS...
Waiting for BigQuery export to complete...
Waiting for BigQuery export to complete...
✓ Export to GCS completed successfully
✓ Export to GCS completed successfully
✓ Download completed
Reading parquet file...
✓ Download completed
Reading parquet file...
Loaded 59170 rows from BigQuery table
Loaded 59170 programs from BigQuery table
Loaded 59170 rows from BigQuery table
Loaded 59170 programs from BigQuery table


In [7]:
from llm_python.datasets.bigquery_converter import convert_bigquery_to_soar, save_soar_parquet

# First, let's inspect the actual data structure
print("Inspecting BigQuery data structure...")
sample_row = raw_data.iloc[0]
print(f"Sample row columns: {sample_row.index.tolist()}")
print(f"Train output type: {type(sample_row['predicted_train_output'])}")
print(f"Train correct type: {type(sample_row['correct_train_input'])}")

print("\n" + "="*50)

# Convert BigQuery data to SOAR format using our reusable function
print("Converting BigQuery data to SOAR format...")
final_dataset = convert_bigquery_to_soar(raw_data, show_progress=True)

# Save the final dataset
if len(final_dataset) > 0:
    output_path = f"/tmp/{file_name}.parquet"
    print(f"Saving final dataset to: {output_path}")
    
    save_soar_parquet(final_dataset, output_path)
else:
    print("No valid data to save!")

Inspecting BigQuery data structure...


NameError: name 'raw_data' is not defined

In [ ]:
# Validate the final dataset using our reusable validation function
from llm_python.datasets.schema import validate_soar_dataset
import pandas as pd

print("=" * 80)
print("DATASET VALIDATION")
print("=" * 80)

file_path = f"/tmp/{file_name}.parquet"

results = validate_soar_dataset(pd.read_parquet(file_path), max_grid_size=40, silent=False)


DATASET VALIDATION
✓ Starting dataset validation...
✓ Dataset shape: (59170, 11)
✓ Unique tasks: 399
✅ Dataset validation passed!
📊 59,170 programs across 399 tasks
🤖 Models: 31
📏 Programs per task: 2-250 (avg: 148.3)

✅ VALIDATION COMPLETE - Dataset is ready for training!
📁 Final dataset location: /tmp/shortest_ratio_2_5x_filtered_250.parquet
❌ VALIDATION FAILED: 'NoneType' object is not subscriptable
✓ Starting dataset validation...
✓ Dataset shape: (59170, 11)
✓ Unique tasks: 399
✅ Dataset validation passed!
📊 59,170 programs across 399 tasks
🤖 Models: 31
📏 Programs per task: 2-250 (avg: 148.3)

✅ VALIDATION COMPLETE - Dataset is ready for training!
📁 Final dataset location: /tmp/shortest_ratio_2_5x_filtered_250.parquet
❌ VALIDATION FAILED: 'NoneType' object is not subscriptable


TypeError: 'NoneType' object is not subscriptable

In [7]:
import duckdb

con = duckdb.connect()
sample_df = con.execute(f"SELECT * FROM '{file_path}' LIMIT 10").fetchdf()
con.close()
print(sample_df)

    task_id reasoning                                               code  \
0  007bbfb7            def transform(grid):\n    pattern = [[0] * 9 f...   
1  007bbfb7            def transform(grid):\n    output = [[0] * 9 fo...   
2  007bbfb7            def transform(grid):\n\n    output = [[0 for _...   
3  007bbfb7            def transform(input_grid):\n    output_grid = ...   
4  007bbfb7            import numpy as np\n\ndef transform(grid_lst: ...   
5  007bbfb7            def transform(grid):\n    n = len(grid)\n    m...   
6  007bbfb7            def transform(grid):\n    output_size = 9\n   ...   
7  007bbfb7            def transform(grid):\n\n    def expand_grid(gr...   
8  007bbfb7            def transform(grid):\n    n = len(grid)\n    n...   
9  007bbfb7            def transform(grid):\n    n = len(grid)\n    n...   

              correct_train_input correct_test_input  \
0  [True, True, True, True, True]             [True]   
1  [True, True, True, True, True]             [True